### 다른 형식으로 만들어보기 위해 만든 테스트용 페이지입니다

In [1]:
from pdfparsing import read_text_file, extract_table
import pandas as pd

In [2]:

df = {}
file_names, desired_table_name_list = read_text_file()
for file_name in file_names:
    df[file_name] = extract_table(file_name, desired_table_name_list)

c:\Users\Owner\anaconda3\envs\pdftorag\lib\site-packages\tabula\io.py:1057: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")
c:\Users\Owner\anaconda3\envs\pdftorag\lib\site-packages\tabula\io.py:1057: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")
c:\Users\Owner\anaconda3\envs\pdftorag\lib\site-packages\tabula\io.py:1057: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")


In [16]:
# 이따하자...

{'samsung': {'연결 재무상태표':                                  과목       2023       2022       2021
  0                                자산        NaN        NaN        NaN
  1                           자산_유동자산  195936557  218470581  218163185
  2                  자산_유동자산_현금및현금성자산   69080893   49680710   39031415
  3                    자산_유동자산_단기금융상품   22690924   65102886   81708986
  4               자산_유동자산_단기상각후원가금융자산     608281     414610    3369034
  5           자산_유동자산_단기당기손익-공정가치금융자산      27112      29080      40757
  6                      자산_유동자산_매출채권   36647393   35721563   40713415
  7                       자산_유동자산_미수금    6633248    6149209    4497257
  8                      자산_유동자산_선급비용    3366130    2867823    2336252
  9                      자산_유동자산_재고자산   51625874   52187866   41384404
  10                   자산_유동자산_기타유동자산    5038838    6316834    5081665
  11                 자산_유동자산_매각예정분류자산     217864          0          0
  12                         자산_비유동자산  259969423  2299

In [15]:
sorted_list = sorted(list(df["hyundai"]["연결 재무상태표"].iloc[:,0]), key = lambda x : x.split("_")[-1])
sorted_list

['자산_비유동자산_공동기업 및 관계기업투자',
 '자산_유동자산_금융업채권',
 '자산_비유동자산_금융업채권',
 '부채_유동부채_기타금융부채',
 '부채_비유동부채_기타금융부채',
 '자산_유동자산_기타금융자산',
 '자산_비유동자산_기타금융자산',
 '부채_유동부채_기타부채',
 '부채_비유동부채_기타부채',
 '자본_지배기업소유주지분_기타자본항목',
 '자산_유동자산_기타자산',
 '자산_비유동자산_기타자산',
 '자산_유동자산_기타채권',
 '자산_비유동자산_기타채권',
 '자본_지배기업소유주지분_기타포괄손익누계액',
 '자산_유동자산_단기금융상품',
 '부채_유동부채_단기차입금',
 '부채_유동부채_당기법인세부채',
 '자산_유동자산_당기법인세자산',
 '부채_유동부채_리스부채',
 '부채_비유동부채_리스부채',
 '부채_유동부채_매각예정비유동부채',
 '자산_유동자산_매각예정비유동자산',
 '부채_유동부채_매입채무',
 '자산_유동자산_매출채권',
 '자산_비유동자산_무형자산',
 '부채_유동부채_미지급금',
 '부채',
 '부채와자본총계',
 '부채_부채총계',
 '부채_비유동부채',
 '자산_비유동자산',
 '자본_비지배지분',
 '자산_비유동자산_사용권자산',
 '부채_비유동부채_사채',
 '부채_비유동부채_순확정급여부채',
 '자산_비유동자산_순확정급여자산',
 '자산_비유동자산_운용리스 자산',
 '부채_유동부채',
 '부채_유동부채_유동성장기부채',
 '자산_유동자산',
 '자산_비유동자산_유형자산',
 '부채_비유동부채_이연법인세부채',
 '자산_비유동자산_이연법인세자산',
 '자본_지배기업소유주지분_이익잉여금',
 '자본',
 '자본_지배기업소유주지분_자본금',
 '자본_지배기업소유주지분_자본잉여금',
 '자본_자본총계',
 '자산',
 '자산_자산총계',
 '자산_비유동자산_장기금융상품',
 '자산_비유동자산_장기성매출채권',
 '부채_비유동부채_장기성미지급금',
 '부채_비유동부채_장기차입금',
 '자산_유동자산_재고자산',
 '